In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [2]:
model= keras.Sequential(
    [
     keras.layers.Dense(8,activation="relu",input_shape=[11]),
     keras.layers.Dense(4,activation="relu"),
     keras.layers.Dense(1,activation="sigmoid")
    ]

)

In [3]:
opt = keras.optimizers.Adam(learning_rate=0.006)
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [4]:
df=pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [5]:
df.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 5000 non-null   int64  
 1   Experience          5000 non-null   int64  
 2   Income              5000 non-null   int64  
 3   Family              5000 non-null   int64  
 4   CCAvg               5000 non-null   float64
 5   Education           5000 non-null   int64  
 6   Mortgage            5000 non-null   int64  
 7   Personal Loan       5000 non-null   int64  
 8   Securities Account  5000 non-null   int64  
 9   CD Account          5000 non-null   int64  
 10  Online              5000 non-null   int64  
 11  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 468.9 KB


In [7]:
df = df[['Age', 'Experience', 'Income', 'Family', 'CCAvg','Education', 'Mortgage', 'Securities Account','CD Account', 'Online', 'CreditCard', 'Personal Loan']]

In [8]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [9]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=69)

In [10]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
x_train.shape, y_train.shape

((3750, 11), (3750,))

In [12]:
model.fit(x_train,y_train,epochs=20,batch_size=64)

Epoch 1/20
59/59 [==============================] - 1s 1ms/step - loss: 0.4264 - accuracy: 0.8629 - precision: 0.2789 - recall: 0.2575
Epoch 2/20
59/59 [==============================] - 0s 1ms/step - loss: 0.1834 - accuracy: 0.9333 - precision: 0.7833 - recall: 0.4356   
Epoch 3/20
59/59 [==============================] - 0s 1ms/step - loss: 0.1288 - accuracy: 0.9536 - precision: 0.8498 - recall: 0.6356
Epoch 4/20
59/59 [==============================] - 0s 1ms/step - loss: 0.0998 - accuracy: 0.9643 - precision: 0.8942 - recall: 0.7178
Epoch 5/20
59/59 [==============================] - 0s 1ms/step - loss: 0.0861 - accuracy: 0.9704 - precision: 0.9150 - recall: 0.7671
Epoch 6/20
59/59 [==============================] - 0s 1ms/step - loss: 0.0782 - accuracy: 0.9715 - precision: 0.9006 - recall: 0.7945
Epoch 7/20
59/59 [==============================] - 0s 1ms/step - loss: 0.0752 - accuracy: 0.9717 - precision: 0.9085 - recall: 0.7890
Epoch 8/20
59/59 [==============================] - 

In [13]:
model.evaluate(x_test,y_test)

40/40 [==============================] - 0s 981us/step - loss: 0.0600 - accuracy: 0.9752 - precision: 0.8962 - recall: 0.8261


[0.06001906096935272,
 0.9751999974250793,
 0.8962264060974121,
 0.8260869383811951]

#PSO

In [14]:
import random
import math
import numpy as np

In [26]:
num_particles = 10
weights_size = model.count_params()

In [27]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes
shape=get_shape(model)
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape)+index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights


In [32]:
class Particle:
    def __init__(self):
        self.position = None
        self.velocity = None
        self.best_position = None
        self.best_fitness = np.inf
    def update_velocity(self, global_best_position, omega=0.5, phi_p=0.5, phi_g=0.5):
        r_p = np.random.rand(weights_size)
        r_g = np.random.rand(weights_size)
        self.velocity = omega*self.velocity + phi_p*r_p*(self.best_position - self.position) + phi_g*r_g*(global_best_position - self.position)
        
    def update_position(self):
        self.position = self.position + self.velocity
        self.position = np.clip(self.position, -1.0, 1.0)
        
def fitness_function(weights):
    model.set_weights(set_shape(weights,shape))
    model.fit(x_train, y_train, epochs=10,verbose=0)
    score = model.evaluate(x_test, y_test)
    return score[1]

In [33]:
particles = [Particle() for _ in range(num_particles)]

In [36]:

for particle in particles:
    particle.position = np.random.uniform(low=-1.0, high=1.0, size=weights_size)
    particle.velocity = np.zeros(weights_size)
    particle.best_position = particle.position.copy()
    particle.best_fitness = np.inf

global_best_position = particles[0].position.copy()
global_best_fitness = fitness_function(global_best_position)

for i in range(1):
    for particle in particles:
        particle.update_velocity(global_best_position)
        particle.update_position()
        fitness = fitness_function(particle.position)
        if fitness < particle.best_fitness:
            particle.best_fitness = fitness
            particle.best_position = particle.position.copy()
        if fitness < global_best_fitness:
            global_best_fitness = fitness
            global_best_position = particle.position.copy()

model.set_weights(set_shape(global_best_position,shape))
score = model.evaluate(x_test, y_test)
print("Final accuracy",score[1])

40/40 [==============================] - 0s 1ms/step - loss: 0.5175 - accuracy: 0.7096 - precision: 0.1575 - recall: 0.4957
Final accuracy 0.7095999717712402
